In [1]:
import sys
sys.path.append("..")
from pathlib import Path
from utils.visualise import get_ocr_stat, display_ocr, display_folder
from utils.config import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_data_train = "../data/train"
path_data_val = "../data/val"
path_data_test = "../data/test"

path_folders_train = "../data/train_folders.txt"
path_folders_val = "../data/val_folders.txt"
path_folders_test = "../data/test_folders.txt"

In [3]:
folders_test = np.array([line.split() for line in open(path_folders_test).readlines(  )])
folders_test

array([['TDJC00734', '1'],
       ['TDJC00982', '1'],
       ['TDJC00458', '1'],
       ...,
       ['TDJC01104', '5000'],
       ['TDJC00093', '5000'],
       ['TDJC01124', '5000']], dtype='<U9')

In [4]:
pages_test_ocr = np.array([(path.stem.split('-') if len(path.stem.split('-'))==2 else [path.stem, '0']) for path in Path(path_data_test).glob("**/*.tsv")])
pages_test_img = np.array([(path.stem.split('-') if len(path.stem.split('-'))==2 else [path.stem, '0']) for path in Path(path_data_test).glob("**/*.jpg")])
assert pages_test_ocr.shape==pages_test_img.shape, f"number of pages in ocr and images not match"

folders_test = np.array([line.split() for line in open(path_folders_test).readlines(  )])
assert np.unique(folders_test[:,0]).shape[0]==np.unique(pages_test_ocr[:,0]).shape[0], f"number of documents in files and txt not match"

print("Test set:")
print(f"   found {np.unique(folders_test[:,0]).shape[0]} documents, {pages_test_ocr.shape[0]} pages, {np.unique(folders_test[:,1]).shape[0]} folders")


Test set:
   found 89 documents, 229 pages, 5000 folders


In [5]:
pages_test_img

array([['TDJC01342', '0'],
       ['TDJC01038', '0'],
       ['TDJC01038', '1'],
       ['TDJC01485', '1'],
       ['TDJC01485', '0'],
       ['TDJC00093', '0'],
       ['TDJC00484', '0'],
       ['TDJC00484', '1'],
       ['TDJC00484', '2'],
       ['TDJC00484', '3'],
       ['TDJC00476', '2'],
       ['TDJC00476', '3'],
       ['TDJC00476', '4'],
       ['TDJC00476', '0'],
       ['TDJC00476', '1'],
       ['TDJC00716', '0'],
       ['TDJC00716', '1'],
       ['TDJC01100', '0'],
       ['TDJC01100', '1'],
       ['TDJC01110', '1'],
       ['TDJC01110', '0'],
       ['TDJC00915', '1'],
       ['TDJC00915', '0'],
       ['TDJC00969', '1'],
       ['TDJC00969', '0'],
       ['TDJC01282', '0'],
       ['TDJC01282', '1'],
       ['TDJC00031', '0'],
       ['TDJC00031', '1'],
       ['TDJC00818', '0'],
       ['TDJC01643', '1'],
       ['TDJC01643', '0'],
       ['TDJC00373', '0'],
       ['TDJC00373', '1'],
       ['TDJC00185', '1'],
       ['TDJC00185', '0'],
       ['TDJC01217', '1'],
 

In [6]:
df = pd.DataFrame({"id": pages_test_img[:,0], "page_num": pages_test_img[:,1]})
df["stem"] = df.id + '-' + df.page_num
df["label"] = (df["page_num"].astype(int)==0).astype(int)

In [7]:
df1 = pd.DataFrame({"id": folders_test[:,0], "folder_id": folders_test[:,1]})

In [8]:
df1 = df1.merge(df, on = "id", how = "left")

In [9]:
folder = df1[["folder_id", "id"]].groupby("folder_id").count()
folder.rename(columns = {"id":"merged_num_of_pages"}, inplace = True)
df1 = df1.merge(folder, on = "folder_id")

In [10]:
df1.head(50)

,id,folder_id,page_num,stem,label,merged_num_of_pages
0,TDJC00734,1,1,TDJC00734-1,0,46
1,TDJC00734,1,2,TDJC00734-2,0,46
2,TDJC00734,1,0,TDJC00734-0,1,46
3,TDJC00982,1,10,TDJC00982-10,0,46
4,TDJC00982,1,12,TDJC00982-12,0,46
5,TDJC00982,1,32,TDJC00982-32,0,46
6,TDJC00982,1,19,TDJC00982-19,0,46
7,TDJC00982,1,16,TDJC00982-16,0,46
8,TDJC00982,1,11,TDJC00982-11,0,46
9,TDJC00982,1,30,TDJC00982-30,0,46


In [11]:
folder.mean()

merged_num_of_pages    28.0458
dtype: float64

In [12]:
df1.to_csv("full_model_prediction.csv")